In [1]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_list_module import *


In [2]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [3]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:2000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(2000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [4]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [5]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [6]:
print(test_query_generator())

[np.float64(25.312893744149584), np.float64(0.46834297686557097), np.float64(0.5709926695728514), np.float64(1.5752251201979668), np.float64(-20.69703581879765), np.float64(0.15850189715330165), np.float64(0.8886825041717856), np.float64(0.02344171273350431), np.float64(0.018903297543244716), np.float64(0.4796584782672293), np.float64(0.8133858509799345), np.float64(103.20940539018895), np.float64(730432.9549014961), np.float64(3.6673764134564535)]


In [7]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [8]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_])
# test_tree.show_root()
with Timer():
    test_tree.build_tree(min_threshold_for_clustering=200)

cur node 0
no. of cur node data 2000
optimal K 2
0's child: [1, 2] 

cur node 1
no. of cur node data 719
optimal K 4
1's child: [3, 4, 5, 6] 

cur node 2
no. of cur node data 1281
optimal K 2
2's child: [7, 8] 

cur node 3
no. of cur node data 238
optimal K 2
3's child: [9, 10] 

cur node 4
no. of cur node data 277
optimal K 3
4's child: [11, 12, 13] 

cur node 5
no. of cur node data 78
added to Popularity min
added to danceability min
added to energy max
added to key max
added to loudness max
added to mode min
added to speechiness min
added to acousticness min
added to instrumentalness max
added to liveness min
added to valence min
added to tempo max
added to duration_in min/ms max
added to time_signature min


cur node 6
no. of cur node data 126
added to energy min
added to key max
added to loudness min
added to mode min
added to speechiness min
added to acousticness max
added to liveness min
added to tempo min
added to duration_in min/ms min
added to time_signature min


cur node 7


In [9]:
print([i.id for i in test_tree.nodes])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


In [10]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [11]:
t = 200000
while t>100:
    seaching = test_query_generator()
    with Timer():
        cur = test_tree.search(seaching)
    t=cur.id
    print(cur.id)

Elapsed time: 0.0008407999994233251 seconds
14


In [12]:
a = test_tree._get_closest_node_within_lv(seaching, cur)
print([n.id for n in a])
print(cur.down_lv)

will return a sorted list of leaf node
[14]
poped 14
cal True False
[]
res [14]
[14]
[]


In [13]:
print(calculation.l1(seaching, a[-1].center))
print(calculation.l1(seaching, a[0].center))

930029.6658931556
930029.6658931556


In [14]:
test_tree.print_tree(test_tree.nodes[0])

└── 0
    ├── 1
    │   ├── 3
    │   │   ├── 9
    │   │   └── 10
    │   ├── 4
    │   │   ├── 11
    │   │   ├── 12
    │   │   └── 13
    │   ├── 5
    │   └── 6
    └── 2
        ├── 7
        │   ├── 14
        │   └── 15
        │       ├── 19
        │       └── 20
        └── 8
            ├── 16
            │   ├── 21
            │   └── 22
            ├── 17
            │   ├── 23
            │   │   ├── 25
            │   │   ├── 26
            │   │   └── 27
            │   └── 24
            │       ├── 28
            │       └── 29
            └── 18


In [15]:
print(test_tree.nodes[0].center)

[0.44036364 0.5291267  0.65660264 0.47645455 0.76121848 0.6405
 0.06197007 0.25581723 0.13625976 0.18137536 0.48564998 0.48213134
 0.18333419 0.728875  ]


In [16]:
res = test_tree.reflection_search(np.mean(normalized_data[:100], axis=0), node=test_tree.nodes[11], degree=2)

1
will return a sorted list of leaf node
[1]
poped 1
[3, 4, 5, 6]
poped 3
poped 4
poped 5
cal True False
poped 6
cal True False
[9, 10, 11, 12, 13]
poped 9
cal True False
poped 10
cal True False
poped 11
cal True False
poped 12
cal True False
poped 13
cal True False
[]
res [10, 9, 13, 11, 12, 5, 6]
will return a one closest node within lv
[1]
poped 1
[3, 4, 5, 6]
poped 3
cal False True
poped 4
cal False True
poped 5
cal True True
poped 6
cal True True
res [3]


In [17]:
res

[[<graph_tree_list_module.Node at 0x2d92af5b450>],
 array([ 0.45109917,  0.57140621,  0.71280317,  0.62084208,  0.78454474,
        -0.72      ,  0.08703692,  0.22744324,  0.12757587,  0.19001313,
         0.49184138,  0.49607945,  0.21332408,  0.75496523]),
 [np.int64(1199),
  np.int64(1582),
  np.int64(1412),
  np.int64(956),
  np.int64(122),
  np.int64(1193),
  np.int64(1532),
  np.int64(1007),
  np.int64(1544),
  np.int64(937)]]

In [18]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(2): 20, np.int32(4): 20, np.int32(6): 14, np.int32(1): 12, np.int32(8): 9, np.int32(5): 7, np.int32(0): 6, np.int32(3): 6, np.int32(7): 5, np.int32(9): 1})
belongs to group:  2 

-- mine result100
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 22, np.int32(6): 11, np.int32(1): 11, np.int32(2): 11, np.int32(5): 10, np.int32(0): 10, np.int32(7): 8, np.int32(9): 7, np.int32(8): 7, np.int32(3): 3})
belongs to group:  4 

-- mine result200
search number:  2000
hitted right result: 100
-- k

Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 24, np.int32(6): 16, np.int32(2): 14, np.int32(8): 14, np.int32(1): 8, np.int32(5): 6, np.int32(0): 6, np.int32(7): 6, np.int32(9): 3, np.int32(3): 3})
belongs to group:  4 

-- mine result400
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 21, np.int32(8): 16, np.int32(1): 14, np.int32(6): 14, np.int32(2): 11, np.int32(5): 10, np.int32(0): 7, np.int32(7): 5, np.int32(3): 2})
belongs to group:  4 

-- mine result500
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, n

In [19]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07]
[0.46450013549986446, 0.40500020499979494, 0.37000020499979497, 0.41500020499979495, 0.40000020499979494, 0.43000020499979497, 0.3850002049997949, 0.41500020499979495, 0.42500020499979496, 0.4645001304998695]
9.99998999939855e-06
4.174001905998094
